In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Conv2DTranspose, Reshape, Dense, Activation, Flatten, \
                                    Dropout, Convolution2D, MaxPooling2D, Input, Resizing, Concatenate

import joblib



In [2]:
data=pd.read_csv('../ckextended.csv')

x_temp=data.pixels
x=np.zeros((920, 2304))
for i in range(920) :
    x[i,:]=x_temp[i].split()
x/=255

x=np.reshape(x, (920, 48,48, 1) )
    
y=data.emotion

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

#On convertit les valeurs de sortie de 0 à 7 en vecteurs
Y_train = tf.keras.utils.to_categorical(y_train, 8)
Y_test = tf.keras.utils.to_categorical(y_test, 8)


In [3]:
inputs = Input(shape=(48,48,1)) 
tens = Convolution2D(32, kernel_size = (3, 3), activation='relu' )(inputs) 
tens = Convolution2D(64, kernel_size = (3, 3), activation='relu' )(tens) 
tens = MaxPooling2D((3, 3))(tens) 
tens = Flatten()(tens) 
tens = Dense(256, activation='relu')(tens) 
tens = Dropout(0.5)(tens) 
outputs= Dense(8, activation='softmax')(tens) 
model = Model(inputs, outputs) 


In [4]:
#Je définis un poids par classe comme la racine de 1/le nombre de cas par classe. J'ai d'abord testé sans racine carrée, 
# mais le poids était trop fort sur les classes les plus rares. 
class_weight={ i : np.sqrt(1/(y_train == i).sum()) for i in range(8)}


In [5]:
batch_size=20
epochs=20
early=tf.keras.callbacks.EarlyStopping(patience=5)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3,
                                                             decay_steps=10000,decay_rate=0.9)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])


history =model.fit(x_train, Y_train, batch_size=batch_size, epochs=epochs,
                   verbose=1,validation_data=(x_test, Y_test), callbacks=[early], class_weight=class_weight)



Epoch 1/20
42/42 [==============================] - 6s 94ms/step - loss: 0.1542 - accuracy: 0.6401 - val_loss: 1.5381 - val_accuracy: 0.6522
Epoch 2/20
42/42 [==============================] - 3s 76ms/step - loss: 0.1283 - accuracy: 0.6969 - val_loss: 0.9464 - val_accuracy: 0.7935
Epoch 3/20
42/42 [==============================] - 3s 77ms/step - loss: 0.0907 - accuracy: 0.7983 - val_loss: 0.6481 - val_accuracy: 0.8370
Epoch 4/20
42/42 [==============================] - 3s 77ms/step - loss: 0.0674 - accuracy: 0.8418 - val_loss: 0.4882 - val_accuracy: 0.9022
Epoch 5/20
42/42 [==============================] - 4s 97ms/step - loss: 0.0568 - accuracy: 0.8418 - val_loss: 0.4191 - val_accuracy: 0.9130
Epoch 6/20
42/42 [==============================] - 4s 84ms/step - loss: 0.0454 - accuracy: 0.8623 - val_loss: 0.3734 - val_accuracy: 0.9239
Epoch 7/20
42/42 [==============================] - 3s 79ms/step - loss: 0.0316 - accuracy: 0.9034 - val_loss: 0.3394 - val_accuracy: 0.9239
Epoch 8/20
42

In [6]:
#sauvegarde du modèle
model_filename = "cnn_emotion.joblib.z"
joblib.dump((model), model_filename)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dropout
......vars
...layers\flatten
......vars
...layers\input_layer
......vars
...layers\max_pooling2d
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-07 09:28:19         4108
metadata.json                                  2023-03-07 09:28:19           64

['cnn_emotion.joblib.z']